In [1]:
import pandas as pd
import numpy as np

from ModelTeacher import ModelTeacher
import Settings
from DatasetPreprocessor import DatasetPreprocessor

In [2]:
md = ModelTeacher()
q = md.load_IDs()

In [3]:
q

array([['00_BN1-129-Eb_comp', 0],
       ['00_BN1-129-Eb_comp', 1],
       ['00_BN1-129-Eb_comp', 2],
       ...,
       ['05_SS3-98-C_solo', 1685],
       ['05_SS3-98-C_solo', 1686],
       ['05_SS3-98-C_solo', 1687]], dtype=object)

In [4]:
csv_file = Settings.ids_path
qq = pd.read_csv(csv_file, header=None)

In [5]:
qq = np.array(pd.read_csv(csv_file, header=None))

In [6]:
qq

array([['00_BN1-129-Eb_comp', 0],
       ['00_BN1-129-Eb_comp', 1],
       ['00_BN1-129-Eb_comp', 2],
       ...,
       ['05_SS3-98-C_solo', 1685],
       ['05_SS3-98-C_solo', 1686],
       ['05_SS3-98-C_solo', 1687]], dtype=object)

In [7]:
qq[0]

array(['00_BN1-129-Eb_comp', 0], dtype=object)

In [8]:
datasetPreprocessor = DatasetPreprocessor()

In [9]:
chunks, frets = datasetPreprocessor.process_audiofile(qq[0][0])

In [10]:
len(chunks)

962

In [11]:
calc_len = len(chunks) // Settings.con_win_size * Settings.con_win_size

In [12]:
chunks = chunks[:calc_len]
print(len(chunks))

954


In [13]:
mt = ModelTeacher()
ids = mt.load_IDs()

In [14]:
ids[1]

array(['00_BN1-129-Eb_comp', 1], dtype=object)

In [15]:
np.array(pd.read_csv(csv_file, header=None))

array([['00_BN1-129-Eb_comp', 0],
       ['00_BN1-129-Eb_comp', 1],
       ['00_BN1-129-Eb_comp', 2],
       ...,
       ['05_SS3-98-C_solo', 1685],
       ['05_SS3-98-C_solo', 1686],
       ['05_SS3-98-C_solo', 1687]], dtype=object)

In [17]:
from sklearn.model_selection import train_test_split
import Settings

In [18]:
X_train, X_test = train_test_split(ids, test_size= (1 - Settings.train_split), random_state=42)

In [19]:
len(X_train)

378048

In [20]:
len(X_test)

94512